In [15]:
import pandas as pd
import quandl, math
import numpy as np
from sklearn import preprocessing, svm
from sklearn import model_selection
from sklearn.linear_model import LinearRegression

import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

import pickle

In [16]:
df = quandl.get('WIKI/GOOGL')
df.to_csv("datasets/WIKI_GOOGL.csv", encoding="utf-8")
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [18]:
df1 = pd.read_csv("datasets/WIKI_GOOGL.csv")
df1.head

<bound method NDFrame.head of             Date     Open     High      Low     Close      Volume  \
0     2004-08-19   100.01   104.06    95.96   100.335  44659000.0   
1     2004-08-20   101.01   109.08   100.50   108.310  22834300.0   
2     2004-08-23   110.76   113.48   109.05   109.400  18256100.0   
3     2004-08-24   111.24   111.60   103.57   104.870  15247300.0   
4     2004-08-25   104.76   108.00   103.88   106.000   9188600.0   
...          ...      ...      ...      ...       ...         ...   
3419  2018-03-21  1092.57  1108.70  1087.21  1094.000   1990515.0   
3420  2018-03-22  1080.01  1083.92  1049.64  1053.150   3418154.0   
3421  2018-03-23  1051.37  1066.78  1024.87  1026.550   2413517.0   
3422  2018-03-26  1050.60  1059.27  1010.58  1054.090   3272409.0   
3423  2018-03-27  1063.90  1064.54   997.62  1006.940   2940957.0   

      Ex-Dividend  Split Ratio    Adj. Open    Adj. High     Adj. Low  \
0             0.0          1.0    50.159839    52.191109    48.12856

In [ ]:
df = df[ ['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_Change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

df_1 = df[ ['Adj. Close', 'HL_PCT', 'PCT_Change', 'Adj. Volume']]
df_1.head()

In [ ]:
import math

forecast_col = 'Adj. Close'
df.fillna(-999999, inplace = True)

forecast_out = int(math.ceil(0.1*len(df)))

df['label'] = df[forecast_col].shift(-forecast_out)
df.head()

In [ ]:
forecast_out = int(math.ceil(0.1*len(df)))

df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
df.head()

In [ ]:
forecast_out

In [ ]:
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
y = np.array(df['label'])
# X = X[:-forecast_out+1]
# OR
# df.dropna(inplace = True) from previous cell
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.5)

clf = LinearRegression(n_jobs = -1)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

In [ ]:
clf = svm.SVR()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

In [ ]:
clf = svm.SVR(kernel = 'poly')
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

In [ ]:
forecast_out = int(math.ceil(0.01*len(df)))
print(forecast_out)
df['label'] = df[forecast_col].shift(-forecast_out)
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]


df.dropna(inplace=True)
y = np.array(df['label'])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)
with open('linearRegression.pickle', 'wb') as f:
    pickle.dump(clf, f)
    
pickle_in = open('linearRegression.pickle', 'rb')
clf = pickle.load(pickle_in)

accuracy = clf.score(X_test, y_test)
accuracy

In [ ]:
forecast_set = clf.predict(X_lately)
print(forecast_set, accuracy, forecast_out)

In [ ]:
df['Forecast'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()